import glob
import json
import re
import matplotlib.pyplot as plt
import numpy as np
import pyhf
from pyhf.contrib.viz import brazil

def rebin(data, newbinsize):
    output = []
    i = 0
    while i < len(data):
        s = 0
        for _ in range(newbinsize):
            if i < len(data):
                s += data[i]
                i += 1
        output.append(s)
    return output

def extract_idx(fname):
    """Parse 'extracted_data_XX.json' and return the integer XX."""
    match = re.search(r"extracted_data_(\d+)\.json", fname)
    return int(match.group(1))

# --------------------------------------------------------------------
# 1) Collect and sort JSON files numerically
# --------------------------------------------------------------------
json_files = glob.glob("extracted_data_*.json")
json_files.sort(key=extract_idx)  # use numeric sort
print("Processing JSON files in numeric order:", json_files)

# Global settings
systematic_value = 0.10
rebinfactor = 10

# (Loop over each file)
for idx, json_file in enumerate(json_files):
    print(f"\n=== Processing {json_file} (idx={idx}) ===")
    with open(json_file, "r") as f:
        data_dict = json.load(f)

    signal_counts = data_dict["hist_photon_energy_signal"]["entries"]
    background_counts = data_dict["hist_photon_energy_BG"]["entries"]

    signal_counts = rebin(signal_counts, rebinfactor)
    background_counts = rebin(background_counts, rebinfactor)

    signal_plus_background = [0.10 * s + b for s, b in zip(signal_counts, background_counts)]
    background_uncertainty = [np.sqrt(b) + systematic_value * b for b in background_counts]

    # Optional: plot the rebinned S and B
    plt.figure(figsize=(8,5))
    plt.plot(signal_counts, label="Signal (rebinned)", alpha=0.7)
    plt.plot(background_counts, label="Background (rebinned)", alpha=0.7)
    plt.xlabel("Rebinned Bins")
    plt.ylabel("Counts")
    plt.title(f"Rebinned S and B for {json_file}")
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"Rebinned_SB_{idx}.png")
    plt.close()

    # Build pyhf model and observations
    model = pyhf.simplemodels.uncorrelated_background(
        signal=signal_counts,
        bkg=background_counts,
        bkg_uncertainty=background_uncertainty,
    )
    observations = signal_plus_background + model.config.auxdata

    # Dynamic POI range helper:
    def dynamic_poi_range(obs_limit, exp_limit, factor=1.5, npoints=10):
        poi_max = factor * max(obs_limit, exp_limit)
        return np.linspace(0.0, poi_max, npoints)

    # Quick upper-limit scan with a default range (e.g., 0-6)
    default_poi_values = np.linspace(0, 6, 10)
    obs_lim_temp, exp_limits_temp, (scan_temp, results_temp) = pyhf.infer.intervals.upper_limits.upper_limit(
        observations, model, default_poi_values, level=0.05, return_results=True
    )

    poi_values = dynamic_poi_range(obs_lim_temp, exp_limits_temp[2], factor=1.5, npoints=10)
    obs_limit, exp_limits, (scan, results) = pyhf.infer.intervals.upper_limits.upper_limit(
        observations, model, poi_values, level=0.05, return_results=True
    )
    
    print(f"Upper limit (obs): μ = {obs_limit:.4f}")
    print(f"Upper limit (exp): μ = {exp_limits[2]:.4f}")

    # Brazil band plot:
    fig, ax = plt.subplots(figsize=(8,5))
    ax.set_title(f"Brazil band for {json_file}")
    brazil.plot_results(poi_values, results, ax=ax)
    plt.tight_layout()
    plt.savefig(f"BrazilBand_{idx}.png")
    plt.close()
    print(f"Created Brazil band plot: BrazilBand_{idx}.png")

print("\nAll files processed.")


In [1]:
import glob, json, re
import matplotlib.pyplot as plt
import numpy as np
import pyhf
from pyhf.contrib.viz import brazil

def parse_index(fname):
    match = re.search(r"extracted_data_(\d+)\.json", fname)
    return int(match.group(1))

json_files = glob.glob("extracted_data_*.json")
json_files.sort(key=parse_index)  # numeric sort

def rebin(data, newbinsize):
    output = []
    i = 0
    while i < len(data):
        s = 0
        for _ in range(newbinsize):
            if i < len(data):
                s += data[i]
                i += 1
        output.append(s)
    return output

for idx, json_file in enumerate(json_files):
    print(f"\n=== Processing file index {idx}, name={json_file} ===")

    # 1) Read the JSON
    with open(json_file,"r") as f:
        data = json.load(f)

    signal_counts = data["hist_photon_energy_signal"]["entries"]
    background_counts = data["hist_photon_energy_BG"]["entries"]


    # 2) Rebin
    signal_counts = rebin(signal_counts, 10)
    background_counts = rebin(background_counts, 10)

     # Debug check
    print("  signal[:5] =", signal_counts[:5], "sum=", sum(signal_counts))
    print("  background[:5] =", background_counts[:5], "sum=", sum(background_counts))

    # 3) S+B
    signal_plus_bkg = [0.1*s + b for s,b in zip(signal_counts, background_counts)]
    bkg_unc = [np.sqrt(b) + 0.1*b for b in background_counts]

    # 4) Build model & fit
    model = pyhf.simplemodels.uncorrelated_background(signal=signal_counts,
                                                      bkg=background_counts,
                                                      bkg_uncertainty=bkg_unc)
    observations = signal_plus_bkg + model.config.auxdata

    # Quick fit
    fit_result = pyhf.infer.mle.fit(observations, model)
    print("  fit result =", fit_result)

    # 5) Quick upper-limit
    poi_vals = np.linspace(0, 10, 10)
    obs_lim, exp_lims, (scan, results) = pyhf.infer.intervals.upper_limits.upper_limit(
        observations, model, poi_vals, level=0.05, return_results=True
    )
    print(f"  obs limit = {obs_lim:.3f}, exp limit central= {exp_lims[2]:.3f}")

    # 6) Plot
    fig, ax = plt.subplots(figsize=(8,5))
    ax.set_title(f"Brazil band for {json_file}")
    brazil.plot_results(poi_vals, results, ax=ax)

    max_limit = max(obs_lim, max(exp_lims))


    xmax = max(6, 1.2 * max_limit)


    ax.set_xlim(0, xmax)
    
    plt.tight_layout()
    
    plt.savefig(f"BrazilBand_{idx}.png")
    plt.close()





AttributeError: 'NoneType' object has no attribute 'group'

# Pseudocode for saving raw data from a TTree:
import json
import ROOT

infile = ROOT.TFile("test_Higgsino_100_50_simple.root")
tree = infile.Get("outputTree")

photon_energies_signal = []
for event in tree:
    for phE in event.ph_E:  # or however your variable is named
        photon_energies_signal.append(phE)

# Save it to JSON
with open("raw_data.json", "w") as f:
    json.dump({"photonE_signal": photon_energies_signal}, f)


import matplotlib.pyplot as plt
import numpy as np

def plot_input_data(
    signal_data,
    background_data,
    variable_name="Photon Energy",
    bin_size=None,
    nbins=None,
    output_name="input_plot.png"
):
    """
    Plot the raw data for signal and background, letting matplotlib do the binning.
    
    Arguments:
        signal_data      : list or np.array of signal values
        background_data  : list or np.array of background values
        variable_name    : string for x-axis label
        bin_size         : if given, we'll build bin edges of that width
        nbins            : if given (and bin_size is None), we let matplotlib do that many bins
        output_name      : name of the output image file
    """
    plt.figure(figsize=(8, 6))

    if bin_size:
        # We'll figure out the range from both datasets
        data_min = min(min(signal_data), min(background_data))
        data_max = max(max(signal_data), max(background_data))
        bin_edges = np.arange(data_min, data_max + bin_size, bin_size)
        plt.hist(signal_data, bins=bin_edges, alpha=0.5, label="Signal")
        plt.hist(background_data, bins=bin_edges, alpha=0.5, label="Background")
    else:
        # If user did not provide bin_size, we can default to nbins or 50
        if not nbins:
            nbins = 50
        plt.hist(signal_data, bins=nbins, alpha=0.5, label="Signal")
        plt.hist(background_data, bins=nbins, alpha=0.5, label="Background")

    plt.xlabel(variable_name)
    plt.ylabel("Counts")
    plt.title("Signal vs Background")
    plt.legend()
    plt.grid(True, axis="y", alpha=0.75)
    plt.tight_layout()
    plt.savefig(output_name)
    plt.close()
    print(f"Saved input plot to {output_name}")


import glob
import json
import matplotlib.pyplot as plt
import numpy as np
import pyhf
from pyhf.contrib.viz import brazil

def rebin(data, newbinsize):
    """
    Combine `newbinsize` consecutive bins from `data` into a single bin.
    E.g., rebinfactor=10 merges groups of 10 bins into 1 bin.
    """
    output = []
    origcount = 0
    while origcount < len(data):
        newbincontent = 0
        for _ in range(newbinsize):
            if origcount < len(data):
                newbincontent += data[origcount]
                origcount += 1
            else:
                break
        output.append(newbincontent)
    return output

def plot_fit_inputs(signal_counts, background_counts, background_uncertainty, rebinfactor, outname):
    """
    Make a plot of rebinned signal vs. background with error bars 
    (showing the sqrt(bkg) + systematic).
    """
    # We'll treat each bin index as the "center" of that bin
    xvals = np.arange(len(signal_counts))
    
    plt.figure(figsize=(8,5))
    
    # Plot background as a marker with error bars
    plt.errorbar(
        xvals, background_counts, 
        yerr=background_uncertainty, 
        fmt='o', color='blue', label='Background'
    )
    
    # Plot signal as a step or line
    plt.step(
        xvals, signal_counts, 
        where='mid', color='red', label='Signal'
    )
    
    plt.xlabel(f"Rebinned bins (factor={rebinfactor})")
    plt.ylabel("Counts")
    plt.title("Input distributions to the fit")
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.legend()
    plt.tight_layout()
    plt.savefig(outname)
    plt.close()


# --------------------------------------------------------------------
# 1. Find all the extracted_data JSON files
# --------------------------------------------------------------------
json_files = glob.glob("extracted_data_*.json")
json_files.sort()  # sort them so they go in numerical order

# We'll keep a 10% systematic
systematic_value = 0.10

# Rebin factor
rebinfactor = 10

for idx, json_file in enumerate(json_files):
    print(f"\n=== Processing {json_file} ===")
    
    # ----------------------------------------------------------------
    # 2. Load data from the JSON file
    # ----------------------------------------------------------------
    with open(json_file, "r") as f:
        data_dict = json.load(f)

    # These keys must match how you stored them in extracted_data_X.json
    signal_counts = data_dict["hist_photon_energy_signal"]["entries"]
    background_counts = data_dict["hist_photon_energy_BG"]["entries"]

    # ----------------------------------------------------------------
    # 3. Rebin signal + background data
    # ----------------------------------------------------------------
    signal_counts = rebin(signal_counts, rebinfactor)
    background_counts = rebin(background_counts, rebinfactor)

    # ----------------------------------------------------------------
    # 4. Define background uncertainties: sqrt(N_bkg) + 10% * N_bkg
    # ----------------------------------------------------------------
    background_uncertainty = [np.sqrt(b) + systematic_value * b for b in background_counts]

    # ----------------------------------------------------------------
    # 5. Plot the input distributions
    # ----------------------------------------------------------------
    input_plot_name = f"input_hist_{idx}.png"
    plot_fit_inputs(
        signal_counts, 
        background_counts, 
        background_uncertainty, 
        rebinfactor, 
        outname=input_plot_name
    )
    print(f"Created input distribution plot: {input_plot_name}")

    # ----------------------------------------------------------------
    # 6. Build the pyhf model
    #    We'll treat 'signal_counts' as the shape of the signal
    #    and 'background_counts' as the shape of the background.
    #    The background_uncertainty is uncorrelated for each bin.
    # ----------------------------------------------------------------
    model = pyhf.simplemodels.uncorrelated_background(
        signal=signal_counts,
        bkg=background_counts,
        bkg_uncertainty=background_uncertainty,
    )

    # ----------------------------------------------------------------
    # 7. Observations = signal_plus_background + auxdata
    #    If you want to see a "signal injection" scenario at 10% of signal
    #    plus background, you can do:
    # ----------------------------------------------------------------
    signal_plus_background = [
        0.10 * s + b for s, b in zip(signal_counts, background_counts)
    ]
    observations = signal_plus_background + model.config.auxdata

    # ----------------------------------------------------------------
    # 8. Fit the model
    # ----------------------------------------------------------------
    fit_result = pyhf.infer.mle.fit(data=observations, pdf=model)
    print(f"Fit result (POI, nuisance params...): {fit_result}")

    # ----------------------------------------------------------------
    # 9. Hypothesis test: compute CLs for mu=1
    # ----------------------------------------------------------------
    CLs_obs, CLs_exp = pyhf.infer.hypotest(
        1.0,  # test mu=1
        observations,
        model,
        test_stat="qtilde",
        return_expected_set=True,
    )
    print(f"Observed CLs: {CLs_obs:.4f}")
    print(f"Expected CLs: {[f'{val:.4f}' for val in CLs_exp]}")
    # ----------------------------------------------------------------
    # 10. Upper-limit scan
    # ----------------------------------------------------------------
    poi_values = np.linspace(0.0, 10, 10)
    obs_limit, exp_limits, (scan, results) = pyhf.infer.intervals.upper_limits.upper_limit(
        observations, model, poi_values, level=0.05, return_results=True
    )
    print(f"Upper limit (obs): μ = {obs_limit:.4f}")
    print(f"Upper limit (exp): μ = {exp_limits[2]:.4f}")

    # ----------------------------------------------------------------
    # 11. Brazil band plot
    # ----------------------------------------------------------------
    brazil_fig_name = f"BrazilBand_{idx}.png"
    fig, ax = plt.subplots(figsize=(8, 5))
    
    ax.set_title(f"Brazil band for {json_file}")
    brazil.plot_results(poi_values, results, ax=ax)
    x_min = poi_values.min() - 1
    x_max = poi_values.max() + 1
    ax.set_xlim(x_min, x_max)

    plt.tight_layout()
    plt.savefig(brazil_fig_name)
    plt.close()
    print(f"Created Brazil band plot: {brazil_fig_name}")

print("\nAll files processed.")
print(f"Expected CLs (exp): {[f'{val:.4f}' for val in CLs_exp]}")





# for row in fit_results:
#     print(f"idx={row['idx']:2d}, m1={row['m1']}, m2={row['m2']},"
#           f" obs={row['obs_mu95']:.3f}, exp={row['exp_mu95']:.3f}")

# # -----------------------------------------------------------------------
# # 5) Build 2D "exclusion" plot
# #    We'll define "excluded" if obs_mu95 < 1, etc.
# # -----------------------------------------------------------------------
# m1vals = np.array([r["m1"] for r in fit_results])
# m2vals = np.array([r["m2"] for r in fit_results])
# obs_vals= np.array([r["obs_mu95"] for r in fit_results])
# exp_vals= np.array([r["exp_mu95"] for r in fit_results])

# excluded_obs = (obs_vals < 1.0).astype(float)  # 1=excluded, 0=not
# excluded_exp = (exp_vals < 1.0).astype(float)

# plt.figure(figsize=(8,6))
# triang = tri.Triangulation(m1vals, m2vals)

# xvals = np.array([r["m1"] for r in fit_results])
# yvals = np.array([r["m2"] for r in fit_results])
# if len(np.unique(xvals)) < 2 or len(np.unique(yvals)) < 2:
#     print("Not enough 2D points to form a Triangulation. Using scatter only.")
#     plt.scatter(xvals, yvals, color="red")
# else:
#     triang = tri.Triangulation(xvals, yvals)
#     plt.tricontourf(triang, excluded_obs, levels=[-0.5,0.5,1.5], colors=["white","red"])


# # Fill with "observed" region => red
# plt.tricontourf(triang, excluded_obs,
#                 levels=[-0.5,0.5,1.5], # 0 -> 0.5 => not excluded, 0.5->1 => excluded
#                 colors=["white","red"], alpha=0.3)
# # Observed boundary in black, solid
# obs_cont = plt.tricontour(triang, excluded_obs, levels=[0.5],
#                           colors=["black"], linestyles=["-"], linewidths=2)

# if obs_cont and len(obs_cont.allsegs[0]) > 0:
#     obs_cont.collections[0].set_label("Observed")


# if exp_cont and len(exp_cont.allsegs[0]) > 0:
#     exp_cont.collections[0].set_label("Expected")

# # Expected boundary in blue, dashed
# exp_cont = plt.tricontour(triang, excluded_exp, levels=[0.5],
#                           colors=["blue"], linestyles=["--"], linewidths=2)
# exp_cont.collections[0].set_label("Expected")
# handles, labels = plt.gca().get_legend_handles_labels()
# if labels:
#     plt.legend(loc="best")
# else:
#     print("No valid contours. Skipping legend.")


# excluded_obs = (obs_vals < 1.0).astype(float)


# plt.xlabel("$m_{\\tilde{\\chi}_2^0}$ [GeV]")
# plt.ylabel("$m_{\\tilde{\\chi}_1^0}$ [GeV]")
# plt.title("FCC-ee SUSY Exclusion @ 95% CL")
# plt.legend(loc="best")

# print("Skipping legend due to degenerate contour.")

# plt.grid(True, alpha=0.4)
# plt.tight_layout()
# plt.savefig("SUSY_Exclusion_2D.png",dpi=150)
# plt.show()

# print("\nFinal 2D plot saved as 'SUSY_Exclusion_2D.png'")

# # Summaries
# for row in fit_results:
#     print(f"idx={row['idx']:2d}, m1={row['m1']}, m2={row['m2']},"
#           f" obs={row['obs_mu95']:.3f}, exp={row['exp_mu95']:.3f}")

# m1vals = np.array([r["m1"] for r in fit_results])
# m2vals = np.array([r["m2"] for r in fit_results])
# obs_vals= np.array([r["obs_mu95"] for r in fit_results])
# exp_vals= np.array([r["exp_mu95"] for r in fit_results])

# excluded_obs = (obs_vals < 1.0).astype(float)  # 1=excluded
# excluded_exp = (exp_vals < 1.0).astype(float)

# plt.figure(figsize=(8,6))

# # If not enough distinct points for a Triangulation, just do scatter:
# if len(np.unique(m1vals))<2 or len(np.unique(m2vals))<2:
#     print("Not enough 2D points to form a Triangulation. Using scatter only.")
#     plt.scatter(m1vals, m2vals, color="red")
# else:
#     triang = tri.Triangulation(m1vals, m2vals)

#     # Fill the plane with white (excluded=0) or red (excluded=1)
#     plt.tricontourf(
#         triang, excluded_obs,
#         levels=[-0.5,0.5,1.5],
#         colors=["yellow","red"],
#         alpha=0.3
#     )

#     # Observed boundary in black
#     obs_cont = plt.tricontour(
#         triang, excluded_obs,
#         levels=[0.5],
#         colors=["black"],
#         linestyles=["-"],
#         linewidths=2
#     )
#     if obs_cont and len(obs_cont.allsegs[0])>0:
#         obs_cont.collections[0].set_label("Observed")

#     # Expected boundary in blue
#     exp_cont = plt.tricontour(
#         triang, excluded_exp,
#         levels=[0.5],
#         colors=["blue"],
#         linestyles=["--"],
#         linewidths=2
#     )
#     # Check if it's empty
#     if exp_cont and len(exp_cont.allsegs[0])>0:
#         exp_cont.collections[0].set_label("Expected")

#     # Build legend only if we have some valid labels
#     handles, labels = plt.gca().get_legend_handles_labels()
#     if labels:
#         plt.legend(loc="best")
#     else:
#         print("No valid contours. Skipping legend.")

# plt.xlabel("$m_{\\tilde{\\chi}_2^0}$ [GeV]")
# plt.ylabel("$m_{\\tilde{\\chi}_1^0}$ [GeV]")
# plt.title("FCC-ee SUSY Exclusion @ 95% CL")
# plt.grid(True, alpha=0.4)
# plt.tight_layout()
# plt.savefig("SUSY_Exclusion_2D.png", dpi=150)
# plt.show()

# print("\nFinal 2D plot saved as 'SUSY_Exclusion_2D.png'")



import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Suppose fit_results is already defined, each element like:
# {
#   "idx": ...,
#   "m1": (heavier mass, e.g. chi2),
#   "m2": (lighter mass, e.g. chi1),
#   "obs_mu95": ...,
#   "exp_mu95": ...
# }

# 1) Print the results
for row in fit_results:
    print(f"idx={row['idx']:2d}, m1={row['m1']}, m2={row['m2']},"
          f" obs={row['obs_mu95']:.3f}, exp={row['exp_mu95']:.3f}")

# 2) Extract mass arrays and upper limits
m1vals = np.array([r["m1"] for r in fit_results])  # e.g. chi2 mass
m2vals = np.array([r["m2"] for r in fit_results])  # e.g. chi1 mass

obs_vals= np.array([r["obs_mu95"] for r in fit_results])
exp_vals= np.array([r["exp_mu95"] for r in fit_results])

# 3) Convert them into x,y for plotting
#    x-axis = m(chi2)
#    y-axis = Δm = m(chi2) - m(chi1)
xvals = m1vals
yvals = m1vals - m2vals  # the mass splitting

# 4) Define excluded vs. allowed
excluded_obs = (obs_vals < 1.0).astype(float)  # 1 => excluded
excluded_exp = (exp_vals < 1.0).astype(float)

plt.figure(figsize=(8,6))

# 5) If not enough distinct points for Triangulation, just do scatter
if len(np.unique(xvals)) < 2 or len(np.unique(yvals)) < 2:
    print("Not enough 2D points to form a Triangulation. Using scatter only.")
    # At least show the points
    plt.scatter(xvals, yvals, color="red")
else:
    # Build triangulation
    triang = tri.Triangulation(xvals, yvals)

    # (A) Fill the plane with two colors:
    #     levels=[-0.5, 0.5, 1.5] => "allowed" if <0.5, "excluded" if >=0.5
    plt.tricontourf(
        triang,
        excluded_obs,
        levels=[-0.5,0.5,1.5],
        colors=["yellow","red"],
        alpha=0.3
    )

    # (B) Observed boundary in black
    obs_cont = plt.tricontour(
        triang,
        excluded_obs,
        levels=[0.5],
        colors=["black"],
        linestyles=["-"],
        linewidths=2
    )
    if obs_cont and len(obs_cont.allsegs[0]) > 0:
        obs_cont.collections[0].set_label("Observed")

    # (C) Expected boundary in blue
    exp_cont = plt.tricontour(
        triang,
        excluded_exp,
        levels=[0.5],
        colors=["blue"],
        linestyles=["--"],
        linewidths=2
    )
    if exp_cont and len(exp_cont.allsegs[0]) > 0:
        exp_cont.collections[0].set_label("Expected")

    # Build legend only if we have valid labels
    handles, labels = plt.gca().get_legend_handles_labels()
    if labels:
        plt.legend(loc="best")
    else:
        print("No valid contours. Skipping legend.")

# 6) Axis labels and finishing touches
plt.xlabel(r"$m(\tilde{\chi}_2^0)\text{ [GeV]}$")
plt.ylabel(r"$\Delta m = m(\tilde{\chi}_2^0)-m(\tilde{\chi}_1^0)$ [GeV]")


plt.title("FCC-ee SUSY Exclusion @ 95% CL")
plt.grid(True, alpha=0.4)
plt.tight_layout()
plt.savefig("SUSY_Exclusion_2D.png", dpi=150)
plt.show()

print("\nFinal 2D plot saved as 'SUSY_Exclusion_2D.png'")



In [4]:
import json
import glob
import numpy as np
import matplotlib.pyplot as plt

# Collect data from all JSON files
json_files = glob.glob("extracted_data_*.json")
data = []

for file in json_files:
    with open(file, 'r') as f:
        data_dict = json.load(f)
        data.append({
            "m_N2": data_dict["m_N2"],
            "delta_m": data_dict["delta_m"],
            "obs_limit": data_dict.get("obs_limit", np.nan),  # Ensure these keys exist
            "exp_limits": data_dict.get("exp_limits", [np.nan]*5)
        })

# Sort data by m_N2
data_sorted = sorted(data, key=lambda x: x["m_N2"])
m_N2_values = [d["m_N2"] for d in data_sorted]
delta_m_values = [d["delta_m"] for d in data_sorted]
obs_limits = [d["obs_limit"] for d in data_sorted]
exp_limits = [d["exp_limits"] for d in data_sorted]

# Extract expected bands
exp_2sigma_low = [e[0] for e in exp_limits]
exp_1sigma_low = [e[1] for e in exp_limits]
exp_median = [e[2] for e in exp_limits]
exp_1sigma_high = [e[3] for e in exp_limits]
exp_2sigma_high = [e[4] for e in exp_limits]

# Create the exclusion plot
plt.figure(figsize=(10, 6))

# Brazil bands
plt.fill_between(m_N2_values, exp_2sigma_low, exp_2sigma_high, color='lime', alpha=0.3, label="Expected ±2σ")
plt.fill_between(m_N2_values, exp_1sigma_low, exp_1sigma_high, color='yellow', alpha=0.5, label="Expected ±1σ")
plt.plot(m_N2_values, exp_median, 'k--', label="Expected median")

# Observed limit
plt.plot(m_N2_values, obs_limits, 'ro-', markersize=5, label="Observed")

# Labels and styling
plt.xlabel(r"$m(\tilde{\chi}_2^0)$ [GeV]", fontsize=12)
plt.ylabel(r"$\Delta m = m(\tilde{\chi}_2^0) - m(\tilde{\chi}_1^0)$ [GeV]", fontsize=12)
plt.title("Higgsino Exclusion Limits (95% CL)", fontsize=14)
plt.legend(loc='upper right', frameon=False)
plt.grid(True, linestyle='--', alpha=0.3)
plt.yscale('log')  # Use if limits span orders of magnitude
plt.xlim(min(m_N2_values)-10, max(m_N2_values)+10)
plt.tight_layout()

# Save and show
plt.savefig("Higgsino_Exclusion_Limits.png", dpi=300)
plt.show()

KeyError: 'm_N2'

In [5]:
import glob
import json
import matplotlib.pyplot as plt
import numpy as np
import pyhf
from pyhf.contrib.viz import brazil

def rebin(data, newbinsize):
    """
    Combine newbinsize consecutive bins from data into a single bin.
    E.g., rebinfactor=10 merges groups of 10 bins into 1 bin.
    """
    output = []
    origcount = 0
    while origcount < len(data):
        newbincontent = 0
        for _ in range(newbinsize):
            if origcount < len(data):
                newbincontent += data[origcount]
                origcount += 1
            else:
                break
        output.append(newbincontent)
    return output

def plot_fit_inputs(signal_counts, background_counts, background_uncertainty, rebinfactor, outname):
    """
    Make a plot of rebinned signal vs. background with error bars 
    (showing the sqrt(bkg) + systematic).
    """
    # We'll treat each bin index as the "center" of that bin
    xvals = np.arange(len(signal_counts))
    
    plt.figure(figsize=(8,5))
    
    # Plot background as a marker with error bars
    plt.errorbar(
        xvals, background_counts, 
        yerr=background_uncertainty, 
        fmt='o', color='blue', label='Background'
    )
    
    # Plot signal as a step or line
    plt.step(
        xvals, signal_counts, 
        where='mid', color='red', label='Signal'
    )
    
    plt.xlabel(f"Rebinned bins (factor={rebinfactor})")
    plt.ylabel("Counts")
    plt.title("Input distributions to the fit")
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.legend()
    plt.tight_layout()
    plt.savefig(outname)
    plt.close()


# --------------------------------------------------------------------
# 1. Find all the extracted_data JSON files
# --------------------------------------------------------------------
json_files = glob.glob("extracted_data_*.json")
json_files.sort()  # sort them so they go in numerical order

# We'll keep a 10% systematic
systematic_value = 0.10

# Rebin factor
rebinfactor = 10

for idx, json_file in enumerate(json_files):
    print(f"\n=== Processing {json_file} ===")
    
    # ----------------------------------------------------------------
    # 2. Load data from the JSON file
    # ----------------------------------------------------------------
    with open(json_file, "r") as f:
        data_dict = json.load(f)

    # These keys must match how you stored them in extracted_data_X.json
    signal_counts = data_dict["hist_photon_energy_signal"]["entries"]
    background_counts = data_dict["hist_photon_energy_BG"]["entries"]

    # ----------------------------------------------------------------
    # 3. Rebin signal + background data
    # ----------------------------------------------------------------
    signal_counts = rebin(signal_counts, rebinfactor)
    background_counts = rebin(background_counts, rebinfactor)

    # ----------------------------------------------------------------
    # 4. Define background uncertainties: sqrt(N_bkg) + 10% * N_bkg
    # ----------------------------------------------------------------
    background_uncertainty = [np.sqrt(b) + systematic_value * b for b in background_counts]

    # ----------------------------------------------------------------
    # 5. Plot the input distributions
    # ----------------------------------------------------------------
    input_plot_name = f"input_hist_{idx}.png"
    plot_fit_inputs(
        signal_counts, 
        background_counts, 
        background_uncertainty, 
        rebinfactor, 
        outname=input_plot_name
    )
    print(f"Created input distribution plot: {input_plot_name}")

    # ----------------------------------------------------------------
    # 6. Build the pyhf model
    #    We'll treat 'signal_counts' as the shape of the signal
    #    and 'background_counts' as the shape of the background.
    #    The background_uncertainty is uncorrelated for each bin.
    # ----------------------------------------------------------------
    model = pyhf.simplemodels.uncorrelated_background(
        signal=signal_counts,
        bkg=background_counts,
        bkg_uncertainty=background_uncertainty,
    )

    # ----------------------------------------------------------------
    # 7. Observations = signal_plus_background + auxdata
    #    If you want to see a "signal injection" scenario at 10% of signal
    #    plus background, you can do:
    # ----------------------------------------------------------------
    signal_plus_background = [
        0.10 * s + b for s, b in zip(signal_counts, background_counts)
    ]
    observations = signal_plus_background + model.config.auxdata

    # ----------------------------------------------------------------
    # 8. Fit the model
    # ----------------------------------------------------------------
    fit_result = pyhf.infer.mle.fit(data=observations, pdf=model)
    print(f"Fit result (POI, nuisance params...): {fit_result}")

    # ----------------------------------------------------------------
    # 9. Hypothesis test: compute CLs for mu=1
    # ----------------------------------------------------------------
    CLs_obs, CLs_exp = pyhf.infer.hypotest(
        1.0,  # test mu=1
        observations,
        model,
        test_stat="qtilde",
        return_expected_set=True,
    )
    print(f"Observed CLs: {CLs_obs:.4f}")
    print(f"Expected CLs: {[f'{val:.4f}' for val in CLs_exp]}")

    # ----------------------------------------------------------------
    # 10. Upper-limit scan
    # ----------------------------------------------------------------
    poi_values = np.linspace(0.0, 10, 10)
    obs_limit, exp_limits, (scan, results) = pyhf.infer.intervals.upper_limits.upper_limit(
        observations, model, poi_values, level=0.05, return_results=True
    )
    print(f"Upper limit (obs): μ = {obs_limit:.4f}")
    print(f"Upper limit (exp): μ = {exp_limits[2]:.4f}")

    # ----------------------------------------------------------------
    # 11. Brazil band plot
    # ----------------------------------------------------------------
    brazil_fig_name = f"BrazilBand_{idx}.png"
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.set_title(f"Brazil band for {json_file}")
    brazil.plot_results(poi_values, results, ax=ax)
    plt.tight_layout()
    plt.savefig(brazil_fig_name)
    plt.close()
    print(f"Created Brazil band plot: {brazil_fig_name}")

print("\nAll files processed.")


=== Processing extracted_data_0(1).json ===
Created input distribution plot: input_hist_0.png
Fit result (POI, nuisance params...): [0.10002243 0.99995972 0.99999856 0.9999988  0.9999988  0.99999872
 0.99999863 0.99999785 0.99999861 0.99999875 0.99999961]


/home/phoenix666/.local/lib/python3.8/site-packages/pyhf/infer/calculators.py:418: RuntimeWarning: divide by zero encountered in scalar divide
  teststat = (qmu - qmu_A) / (2 * self.sqrtqmuA_v)
/home/phoenix666/.local/lib/python3.8/site-packages/pyhf/infer/calculators.py:467: RuntimeWarning: invalid value encountered in divide
  CLs = tensorlib.astensor(CLsb / CLb)


Observed CLs: nan
Expected CLs: ['1.0000', '1.0000', '1.0000', '1.0000', '1.0000']
Upper limit (obs): μ = nan
Upper limit (exp): μ = 2.1667
Created Brazil band plot: BrazilBand_0.png

=== Processing extracted_data_0.json ===
Created input distribution plot: input_hist_1.png
Fit result (POI, nuisance params...): [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Observed CLs: 0.5000
Expected CLs: ['0.0000', '0.0000', '0.0000', '0.0000', '0.0000']


/usr/local/lib/python3.8/dist-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Upper limit (obs): μ = 2.1111
Upper limit (exp): μ = 1.0556
Created Brazil band plot: BrazilBand_1.png

=== Processing extracted_data_1(1).json ===
Created input distribution plot: input_hist_2.png
Fit result (POI, nuisance params...): [0.09985837 1.00011586 1.00000578 1.00000591 1.0000061  1.00000875
 1.00000928 1.00000959 1.00001201 1.0000093  1.00000038]
Observed CLs: 0.0000
Expected CLs: ['0.0000', '0.0000', '0.0000', '0.0000', '0.0000']
Upper limit (obs): μ = 1.0556
Upper limit (exp): μ = 1.0556
Created Brazil band plot: BrazilBand_2.png

=== Processing extracted_data_1.json ===
Created input distribution plot: input_hist_3.png
Fit result (POI, nuisance params...): [0.09985837 1.00011586 1.00000578 1.00000591 1.0000061  1.00000875
 1.00000928 1.00000959 1.00001201 1.0000093  1.00000038]
Observed CLs: 0.0000
Expected CLs: ['0.0000', '0.0000', '0.0000', '0.0000', '0.0000']
Upper limit (obs): μ = 1.0556
Upper limit (exp): μ = 1.0556
Created Brazil band plot: BrazilBand_3.png

=== Pro